In [1]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
#globals
equities = ['SPY', 'IWM', 'QQQ', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR']
fixed_income = ['TLT', 'IEF']
precious_metals = ['GLD', 'GDX', 'SLV', 'GDXJ']

tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

In [3]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

def get_benchmark(indices,start, end):
    indices = indices

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

def ticker_data(tickers, start, end):
    days = (end-start).days

    index = pd.date_range(start, periods = days, freq='D')
    df = pd.DataFrame(index = index,columns = tickers)

    for ticker in tickers:
        df[ticker] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    df = df.dropna()
    return(df)

def portfolio_data(portfolio, start, end, rf, leverage, margin_int):
    
    weights = np.array(list(portfolio.values()))
    tickers = list(portfolio.keys())
    data = ticker_data(tickers, start, end)
    mean_returns = data.pct_change().mean()
    cov = data.pct_change().cov()
    
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std

    portfolio_stats = {'ret': portfolio_return, 'stdev':portfolio_std, 'sharpe': sharpe_ratio,
                      'levered ret': levered_return, 'levered stdev':levered_std, 'levered sharpe': levered_sharpe}
    
    portfolio_df = pd.DataFrame(portfolio_stats,columns=['ret','stdev','sharpe'] +list(portfolio.keys())+ ['levered ret','levered stdev','levered sharpe'], index = [0])
    
    for ticker in tickers:
        portfolio_df[ticker] = portfolio[ticker]
    
    return(portfolio_df)

In [4]:
def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df

#simulate leveraged portfolios

def calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std
    
    return portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe

def simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int):
    results_matrix = np.zeros((len(mean_returns)+6, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe = calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = levered_return
        results_matrix[4,i] = levered_std
        results_matrix[5,i] = levered_sharpe
        
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+6,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe','levered ret','levered stdev','levered sharpe'] + [ticker for ticker in tickers])
        
    return results_df

In [8]:
#find target leverage

def apply_leverage(input_portfolio, leverage, margin_int):
    
    results_frame = input_portfolio.copy()
    results_frame['levered ret']=[0.0]*len(results_frame)
    results_frame['levered stdev']=[0.0]*len(results_frame)
    results_frame['levered sharpe'] = [0.0]*len(results_frame)
    
    for i in range(len(results_frame)):
        results_frame['levered ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
        results_frame['levered stdev'][i] = results_frame['stdev'][i] * leverage
        results_frame['levered sharpe'][i] = results_frame['levered ret'][i]/results_frame['levered stdev'][i]
        
    return(results_frame)

def target_risk(portfolios, acceptable_risk, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    leverage = float(acceptable_risk / optimal_portfolio['stdev'])
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def target_return(portfolios, tgt_rtn, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    
    leverage = (tgt_rtn - margin_int)/(float(optimal_portfolio['ret']-margin_int))
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)

In [6]:
indices = ['SPY', 'IWM', 'VT', 'VEF.TO', 'VTI']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
standard = get_benchmark(indices,start, end)
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104362,0.199618,0.522807
2,IWM,0.0972809,0.248786,0.391022
3,VT,0.0850438,0.22475,0.378392
4,VEF.TO,0.0907691,0.168605,0.538353
5,VTI,0.105993,0.201052,0.527193


In [11]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
ticker_df = ticker_data(tickers, start, end)
ticker_df.head()

,SPY,GLD,TLT,LQD
2005-07-25,90.983475,42.509998,56.903557,59.293636
2005-07-26,91.094292,42.250000,56.995152,59.304337
2005-07-27,91.426628,42.439999,56.903557,59.229378
2005-07-28,92.002747,42.750000,57.410233,59.454315
2005-07-29,91.389702,42.820000,56.842503,59.320442


In [44]:
#IB
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.015

portfolio_DF = portfolio_data(portfolio, start, end, rf, leverage, margin_int)
portfolio_DF

,ret,stdev,sharpe,TLT,SPY,GLD,LQD,levered ret,levered stdev,levered sharpe
0,0.089808,0.079761,1.125968,0.402064,0.293876,0.157595,0.146465,0.127212,0.119641,1.06328


In [43]:
#questrade
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.07

portfolio_DF = portfolio_data(portfolio, start, end, rf, leverage, margin_int)
portfolio_DF

,ret,stdev,sharpe,TLT,SPY,GLD,LQD,levered ret,levered stdev,levered sharpe
0,0.089808,0.079761,1.125968,0.402064,0.293876,0.157595,0.146465,0.099712,0.119641,0.833426


In [26]:
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}

weights = np.array(list(portfolio.values()))
tickers = list(portfolio.keys())

data = ticker_data(tickers, start, end)
mean_returns = data.pct_change().mean()
cov = data.pct_change().cov()
portfolio_return = np.sum(mean_returns * weights) * 252

0.0898079574485952

In [13]:
tickers = ['VGT','VTI', 'GLD', 'TLT', 'LQD', 'IEF']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
results_frame.head()

,ret,stdev,sharpe,VGT,VTI,GLD,TLT,LQD,IEF
0,0.126944,0.165411,0.767450,0.464538,0.311924,0.170030,0.015814,0.022571,0.015123
1,0.103594,0.100138,1.034511,0.255473,0.249161,0.110623,0.162826,0.049633,0.172285
2,0.099610,0.082528,1.206977,0.291100,0.035488,0.191395,0.102352,0.179874,0.199791
3,0.109409,0.098987,1.105280,0.301499,0.201350,0.077053,0.362184,0.023059,0.034855
4,0.098998,0.102813,0.962892,0.262723,0.208384,0.111255,0.010141,0.232778,0.174718


In [92]:
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['sharpe']==max_sharpe]

holdings = {'VGT':0.174358, 'VTI':0.026624, 'GLD':0.140878, 'USO':0.012532, 'LQD':0.050989, 'IEF':0.45764}

,ret,stdev,sharpe,VGT,VTI,GLD,USO,TLT,LQD,IEF
9545,0.082961,0.063847,1.299372,0.174358,0.026624,0.140878,0.012532,0.136978,0.050989,0.45764


In [414]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

df = ticker_data(tickers, start, end)
num_portfolios = 10000
mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()

leveraged_results = simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int)
leveraged_results.head()

,ret,stdev,sharpe,levered ret,levered stdev,levered sharpe,SPY,GLD,TLT,LQD
0,0.092329,0.097367,0.948250,0.128493,0.146051,0.879781,0.120488,0.323986,0.494597,0.060929
1,0.086349,0.088262,0.978326,0.119524,0.132394,0.902794,0.140439,0.357289,0.109425,0.392848
2,0.093409,0.107904,0.865674,0.130114,0.161856,0.803891,0.426877,0.285155,0.018070,0.269898
3,0.096476,0.108936,0.885625,0.134714,0.163404,0.824427,0.433431,0.320651,0.063359,0.182560
4,0.091822,0.083633,1.097915,0.127732,0.125449,1.018201,0.305456,0.238433,0.294742,0.161369


In [418]:
max_lev_sharpe = leveraged_results['levered sharpe'].max()
leveraged_results[leveraged_results['levered sharpe']==max_lev_sharpe]

,ret,stdev,sharpe,levered ret,levered stdev,levered sharpe,SPY,GLD,TLT,LQD
5743,0.090494,0.080496,1.124203,0.125741,0.120744,1.041383,0.296776,0.176891,0.391048,0.135285


In [19]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0

portfolios = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

margin_int = 0.02
risk = float(standard[standard['benchmark']=='SPY']['stdev'])*0.8
opt_levered_portfolio = target_risk(portfolios, risk, start, end, margin_int)
opt_levered_portfolio

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ret,stdev,sharpe,SPY,GLD,TLT,LQD,levered ret,levered stdev,levered sharpe,leverage
806,0.090014,0.080044,1.124557,0.294121,0.169345,0.391702,0.144831,0.159666,0.159675,0.999948,1.994849


In [24]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0

portfolios = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

margin_int = 0.02
tgt_rtn = 0.14

opt_levered_portfolio = target_return(portfolios, tgt_rtn, start, end, margin_int)
opt_levered_portfolio

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,ret,stdev,sharpe,SPY,GLD,TLT,LQD,levered ret,levered stdev,levered sharpe,leverage
8854,0.089949,0.079992,1.124477,0.293877,0.1731,0.380804,0.152219,0.14,0.137229,1.020194,1.715533


In [476]:
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104145,0.199594,0.521784
2,IWM,0.0968185,0.24876,0.389205
3,VT,0.0850438,0.22475,0.378392
4,QQQ,0.157312,0.214189,0.734452


In [ ]:
def portfolio_stats(portfolio, start, end, rf, leverage, margin_int):
    
    weights = np.array(list(portfolio.values()))
    tickers = list(portfolio.keys())
    data = ticker_data(tickers, start, end)
    mean_returns = data.pct_change().mean()
    cov = data.pct_change().cov()
    
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std

    portfolio_stats = {'ret': portfolio_return, 'stdev':portfolio_std, 'sharpe': sharpe_ratio,
                      'levered_ret': levered_return, 'levered_std':levered_std, 'levered_sharpe': levered_sharpe}
    
    return(portfolio_stats)

def simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int):
    results_matrix = np.zeros((len(mean_returns)+6, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe = calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = levered_return
        results_matrix[4,i] = levered_std
        results_matrix[5,i] = levered_sharpe
        
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+6,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe','levered ret','levered stdev','levered sharpe'] + [ticker for ticker in tickers])

In [40]:
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

stats = portfolio_stats(portfolio, start, end, rf, leverage, margin_int)

In [48]:
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

my_portfolio = portfolio_stats(portfolio, start, end, rf, leverage, margin_int)

In [51]:
my_portfolio

,ret,stdev,sharpe,TLT,SPY,GLD,LQD,levered ret,levered stdev,levered sharpe
0,0.089645,0.079753,1.124037,0.402064,0.293876,0.157595,0.146465,0.124467,0.119629,1.040445


In [ ]:
# risk allocation to capital allocation
# risk allocation